In [ ]:
from datetime import datetime
import os
import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset
import pandas as pd
from netCDF4 import num2date
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import time
import sys
import cmaps
sys.path.append(r"/home/yfdong/data/work/LF-SAM/code/Library")
from MeteoVarPlot import (add_quiverkey,
                          draw_2DVAR, draw_Bias, draw_VARtemporal, draw_Bias_subplot,draw_Prec_subplot ,
                          draw_2DuvVAR ,draw_uvBias_subplot ,draw_2DuvVAR_subplot
                        #   ,draw_Prec_temporal_subplot
                          )
from PreprocessVar import (
    get_MONTH_abbr, get_pentad, get_letter,extract_SubsetRegion_mask,
    preprocess_var, read_var,
    Read_VarsFromVarsDict, experiments_types,
    load_geodata, SubsetDomainConfigure ,create_domain_mask,get_letter, Meiyu_configs, 
    JJA_configs,Events_configs,get_letter,
    return_Event_Date,LST_Convert_to_UTC,get_indices_for_dates,save_CHM_PRE_daily,
    Events_configs_total ,getFileList,create_nc_file,read_cmorph_var,save_cmorph_hourly,calculate_metric
)

In [ ]:
# 读取数据
import xarray as xr
# 读取数据
BasePath = "/home/yfdong/data/work/LF-SAM/"
DataPath = '/raid61/yfdong/data/work/LF-SAM'
CalibrationStatus = "def" 
VAR_NAME = 'RAINNC'
# 定义结果字典
years = [2011, 2018, 2020]

# 初始化结果字典
results_src_grid = {}
# 遍历每个年份
for year in years:
    # 为每个年份创建一个包含三个实验的字典
    results_src_grid[year] = {
        'WRF-S': {},
        'WRF-H': {},
        'CHM_PRE': {},
        'CMORPH': {}
    }
# 读取地形数据
Ctrl_lat, Ctrl_lon, LandMask, DEM = load_geodata(GeoPath ="/raid61/yfdong/data/work/LF-SAM/Domain/def/geo_em.nc",IF_LandMask=True)
experiment = experiments_types[0]
# Events_config = JJA_configs[1]
for Events_config in JJA_configs:
    EventName = Events_config['EventName']
    print(Events_config)
    EventState='Total'
    Timedelta = 24 # Hours
    LAYER, SCALE_FACTOR = 0, 1
    # 读取事件的开始和结束时间
    ReadStartDate, ReadEndDate = return_Event_Date(EventState, Events_config)
    Year = ReadStartDate.year
    DateList = pd.date_range(start=ReadStartDate, end=ReadEndDate, freq=f'{Timedelta}h')

    # 读取WRF-S模拟数据
    DATA_PATH = os.path.join(DataPath, f'output/JJA/{EventName}/nc')
    results_src_grid[Year]['WRF-S'],_ = preprocess_var(DATA_PATH, CalibrationStatus, experiment['control_method'], 
                    VAR_NAME, DateList, ReadStartDate, ReadEndDate, LAYER, 
                    SCALE_FACTOR, LandMask, IF_LANDMASK=False)
    
    # 读取WRF-H模拟数据
    results_src_grid[Year]['WRF-H'],_ = preprocess_var(DATA_PATH, CalibrationStatus, experiment['experiment_method'], 
                    VAR_NAME, DateList, ReadStartDate, ReadEndDate, LAYER, 
                    SCALE_FACTOR, LandMask, IF_LANDMASK=False)   
     
    # 读取CHM_PRE数据
    dstFilePath = os.path.join(DataPath, f'obs/CHM_PRE/JJA_{Year}_CHM_PRE_1-Daily.nc')
    FilePath = dstFilePath   
    results_src_grid[Year]['CHM_PRE'], CHM_lat, CHM_lon = read_cmorph_var(dstFilePath, ReadStartDate, ReadEndDate, 'Prec' ,SCALE_FACTOR )

    # 读取CMORPH数据
    dstFilePath = os.path.join(DataPath, f'obs/CMORPH/JJA_{Year}_cmorph_1-Hourly.nc')
    Var, CMORPH_lat, CMORPH_lon = read_cmorph_var(dstFilePath, ReadStartDate, ReadEndDate, 'Prec' ,SCALE_FACTOR )
    # -------------------- CMORPH 转化为daily数据 ------------------
    CMORPH_DateList = pd.date_range(start=f'{year}-06-01', end=f'{year}-08-30', freq='1h')
    # 创建 xarray Dataset
    ds = xr.Dataset(
        {
            "Prec": (["time", "lat", "lon"], Var)
        },
        coords={
            "time": CMORPH_DateList,
            "lat": CMORPH_lat,
            "lon": CMORPH_lon
        }
    )
    # 使用 resample 方法计算日均值
    daily_ds = ds.resample(time="1D").mean()
    # 提取日均值数据
    results_src_grid[Year]['CMORPH'] = daily_ds['Prec'].values
    
    
    

In [ ]:
# 统一分辨率
'''
To assess the performance of precipitation simulations, 
both the CHM_PRE and WRF-S model outputs were spatially interpolated onto the CMORPH grid using a bilinear interpolation technique
Observation data: CHM_PRE and CMORPH, while the WRF-S is model output.
'''
from scipy.interpolate import griddata, RegularGridInterpolator
# 初始化结果字典
results_CMORPH_grid = {}
# 遍历每个年份
for year in years:
    # 为每个年份创建一个包含三个实验的字典
    results_CMORPH_grid[year] = {
        'WRF-S': {},
        'CHM_PRE': {},
        'CMORPH': {}
    }


# print(test.shape,src_lat, src_lon)
# --------------------- 提取子区域 --------------------
def create_domain_mask(lat, lon, domain):
   domain_lat_min, domain_lat_max, domain_lon_min, domain_lon_max = domain
   mask = (lat >= domain_lat_min) & (lat <= domain_lat_max) & (lon >= domain_lon_min) & (lon <= domain_lon_max)
   masked_array = xr.DataArray(np.where(mask, 1, np.nan), dims=[ 'south_north', 'west_east'])
   return masked_array
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
DomainRegion = [25, 35, 108, 122] # 计算纬向平均的经纬度范围
dst_lon_grid , dst_lat_grid= np.meshgrid(CMORPH_lon, CMORPH_lat)
DomainMask = create_domain_mask(dst_lat_grid, dst_lon_grid, DomainRegion)
# 提取子区域
DomainMask = np.expand_dims(DomainMask,0) #广播
print(dst_lat_grid.shape, dst_lon_grid.shape)
# -------------------- 统一分辨率 ----------------------------
def interpolate_to_cmorph(src_var, src_lat, src_lon, CMORPH_lat, CMORPH_lon):
    
    dst_lat, dst_lon = CMORPH_lat, CMORPH_lon
    dst_lon_grid, dst_lat_grid = np.meshgrid(dst_lon, dst_lat)
    dst_points = np.array([dst_lat_grid.flatten(), dst_lon_grid.flatten()]).T
    
    # 强制类型转换
    dst_points = dst_points.astype(np.float32)
    REGRIDmethod = "linear"
    REGRIDmethod = "nearest"
    if src_var.shape[0]== 1 or src_var.ndim==2:
        dst_var = np.zeros((int(len(dst_lat)), int(len(dst_lon))))
        print('SRC',src_var.shape,src_lat.shape, src_lon.shape)
        print('dst',dst_var.shape, dst_lat.shape, dst_lon.shape)
        interpolator = RegularGridInterpolator((src_lat, src_lon), src_var, method=REGRIDmethod,bounds_error=False, fill_value=None)
        dst_var[:] = interpolator(dst_points).reshape((len(dst_lat), len(dst_lon)))
    elif src_var.shape[0] > 1 or src_var.ndim>=2:
        dst_var = np.zeros((int(src_var.shape[0]), int(len(dst_lat)), int(len(dst_lon))))
        for DateIndex in range(len(DateList)):  
            interpolator = RegularGridInterpolator((src_lat, src_lon), src_var[DateIndex,:], method=REGRIDmethod,bounds_error=False, fill_value=None)
            dst_var[DateIndex,:] = interpolator(dst_points).reshape((len(dst_lat), len(dst_lon)))
    else:
        print('Error: src_var.shape[0] is not 1 or greater than 1')
    return dst_var 
    
years = [2011, 2018, 2020]
Vars = ['WRF-S', 'WRF-H','CHM_PRE']
for yearIndex, year in enumerate(years):
    for varIndex, var in enumerate(Vars):
        print(year, var)
        # 读取源数据
        src_var = results_src_grid[year][var]
        if var=='WRF-S' or var=='WRF-H':
            src_lon, src_lat = Ctrl_lon[1,:], Ctrl_lat[:,1]
        elif var=='CHM_PRE':
            src_lon, src_lat = CHM_lon, CHM_lat
        # 进行插值计算
        interpolate_data= interpolate_to_cmorph(src_var, src_lat, src_lon, CMORPH_lat, CMORPH_lon)
        interpolate_data[interpolate_data>1000]=np.nan
        interpolate_data[interpolate_data<0]=np.nan
        results_CMORPH_grid[year][var] = interpolate_data *DomainMask
    results_CMORPH_grid[year]['CMORPH'] = results_src_grid[year]['CMORPH'] *DomainMask
# 插值地形
dstDEM = interpolate_to_cmorph(DEM, Ctrl_lat[:,1], Ctrl_lon[1,:], CMORPH_lat, CMORPH_lon)
dstLandMask = interpolate_to_cmorph(LandMask, Ctrl_lat[:,1], Ctrl_lon[1,:], CMORPH_lat, CMORPH_lon)
        # # 计算均方根误差（RMSE）
        # RMSE = np.sqrt(np.mean((results_CMORPH_grid[year]['WRF-S'] - results_CMORPH_grid[year]['CMORPH'])**2))
        # # 计算均方根误差（RMSE）
        # RMSE = np.sqrt(np.mean((results_CMORPH_grid[year]['WRF-S] - results_CMORPH_grid[year]['CHM_PRE'])**2))
        # # 计算相关系数
        # CORR = np.corrcoef(results_CMORPH_grid[year]['WRF-S'].flatten(), results_CMORPH_grid[year]['CMORPH'].flatten())[0, 1]
        # # 计算相关系数
        # CORR = np.corrcoef(results_CMORPH_grid[year]['CHM_PRE'].flatten(), results_CMORPH_grid[year]['CMORPH'].flatten())[0, 1]


In [ ]:
# 散点图——皖南山区
import xarray as xr
import matplotlib.dates as mdates
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.patches import Rectangle

# --------------------- 提取子区域 --------------------
def create_domain_mask(lat, lon, domain):
   domain_lat_min, domain_lat_max, domain_lon_min, domain_lon_max = domain
   mask = (lat >= domain_lat_min) & (lat <= domain_lat_max) & (lon >= domain_lon_min) & (lon <= domain_lon_max)
   masked_array = xr.DataArray(np.where(mask, 1, np.nan), dims=[ 'south_north', 'west_east'])
   return masked_array
def extract_SubsetRegion_mask(var ,domain_mask):
    row_indices, col_indices = np.where(domain_mask == 1)  # 提取 ==1 的区域
    row_start, row_end = row_indices.min(), row_indices.max()
    col_start, col_end = col_indices.min(), col_indices.max()
    # 步骤2：切片获取子区域数据（保持二维结构）
    sub_var = var[row_start:row_end+1, col_start:col_end+1]
    return sub_var
DomainRegion = [28.7172627, 30.7172627, 116.8324811, 119.8324811] # 计算纬向平均的经纬度范围
# DomainRegion = [24, 38, 109, 122]
dst_lon_grid , dst_lat_grid= np.meshgrid(CMORPH_lon, CMORPH_lat)
DomainMask = create_domain_mask(dst_lat_grid, dst_lon_grid, DomainRegion)
# 提取子区域
DomainMask = np.expand_dims(DomainMask,0) #广播

PrecAeraMeanVar_df = pd.DataFrame({
    'Date': [],
    'CMORPH': [],
    'CHM_PRE': [], 
    'WRF-S': [], 
    'WRF-H': [], 
})
for year in [2011, 2018, 2020]:
    template_df = pd.DataFrame({
        'Date': [],
        'CMORPH': [],
        'CHM_PRE': [], 
        'WRF-S': [],
        'WRF-H': [],
    })
    for var in ['CMORPH', 'CHM_PRE', 'WRF-S', 'WRF-H']:
        template_df[var] = pd.Series(np.nanmean(results_CMORPH_grid[year][var]*DomainMask, axis =(1,2)))
    template_df['Date'] = pd.date_range(start=f'{year}-06-01', end=f'{year}-08-30', freq='D')
    PrecAeraMeanVar_df = pd.concat([PrecAeraMeanVar_df, template_df])
PrecAeraMeanVar_df['Date'] = pd.to_datetime(PrecAeraMeanVar_df['Date'])
PrecAeraMeanVar_df.columns
PrecAeraMeanVar_df['CMORPH'] = PrecAeraMeanVar_df['CMORPH'] * 24 
print(PrecAeraMeanVar_df.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append("/home/yfdong/data/work/LF-SAM/code/Library")
from MeteoChartPlot import (plot_AWB_DIFF , plot_AWB_Mean ,plot_TWB_DIFF ,
                            plot_TWB_Mean , preprocess_WB_Df, add_TickGrid)
def get_letter(i):
    if 0 <= i < 26:
        return chr(ord('a') + i)
    else:
        return None
# PrecAeraMeanVar_df =PrecAeraMeanVar_df[PrecAeraMeanVar_df['Date'].dt.year==2011]
# 创建子图
from scipy.stats import pearsonr  # 用于计算Pearson相关系数
def calculate_rmse_corr(observed, modeled):
    """
    计算模型输出与观测值之间的RMSE和相关系数。
    
    参数:
        observed (np.array): 观测值数组。
        modeled (np.array): 模型预测值数组。
        
    返回:
        rmse (float): RMSE值。
        corr (float): 相关系数值。
    """
    rmse = np.sqrt(np.mean((observed - modeled) ** 2))
    corr, _ = pearsonr(observed, modeled)
    rmae = np.mean(np.abs(observed - modeled))/np.mean(observed) *100
    bias = np.mean(modeled-observed)
    return rmse, corr, rmae, bias
# 创建空的DataFrame来存储结果
results = []

PlotMax = 30
PlotMax = 100
fig, axs = plt.subplots(3, 2, figsize=(14/1.5, 5*3/1.5), gridspec_kw={'width_ratios': [3, 6]}, sharey=True, dpi=500)
for YearIndex, Year in enumerate([2011,2018,2020]):
    ax1 = axs[YearIndex, 0]
    ax2 = axs[YearIndex, 1]
    Sub_df = PrecAeraMeanVar_df[PrecAeraMeanVar_df['Date'].dt.year==Year]
    # 绘制左侧散点图
    ax1.scatter(Sub_df['CMORPH'], Sub_df['WRF-S'], linewidths=1.5, edgecolor="#FD4302", facecolor='#FFF0F5', s=80, alpha=0.8, label='WRF-S')
    ax1.scatter(Sub_df['CMORPH'], Sub_df['WRF-H'], linewidths=1.5, edgecolor="#0C87FF", facecolor='#BFEFFF', s=80, alpha=0.8, label='WRF-H')
    ax1.set_xlim(0, PlotMax)
    ax1.set_ylim(0, PlotMax)
    ax1.set_xlabel('CMORPH (mm/day)')
    ax1.set_ylabel('Sim (mm/day)')
    ax1.legend(loc = 'upper right')
    add_TickGrid(ax1, IFminorOn=True)
    # ------------------- 计算评分：-------------------------
    Sub_df = PrecAeraMeanVar_df[PrecAeraMeanVar_df['Date'].dt.year == Year]
    # 对于WRF-S
    rmse_wrf_s, corr_wrf_s, rmae_wrf_s, bias_wrf_s = calculate_rmse_corr(Sub_df['CMORPH'][0:-1], Sub_df['WRF-S'][0:-1])
    results.append({'Year': Year, 
                    'Model': 'WRF-S', 
                    'RMSE': rmse_wrf_s, 
                    'CORR': corr_wrf_s,
                    'BIAS': bias_wrf_s,
                    'RMAE': rmae_wrf_s},
                   )
    # 对于WRF-H
    rmse_wrf_h, corr_wrf_h, rmae_wrf_h, bias_wrf_h = calculate_rmse_corr(Sub_df['CMORPH'][0:-1], Sub_df['WRF-H'][0:-1])
    results.append({'Year': Year, 
                    'Model': 'WRF-H', 
                    'RMSE': rmse_wrf_h, 
                    'CORR': corr_wrf_h,
                    'BIAS': bias_wrf_h,
                    'RMAE': rmae_wrf_h}, 
                   )

    ax1.text(0.025, 0.95, f"({get_letter(YearIndex+0)})", fontsize=14, transform=ax1.transAxes, va='top')
    # 添加对角线
    ax1.plot([0, PlotMax], [0, PlotMax], color='black', linestyle='--', linewidth=1)
    
    # 绘制右侧折线图
    ax2.plot(Sub_df['Date'], Sub_df['CMORPH'], color='gray', marker='o', linestyle='None', label='CMORPH', alpha=0.6)
    ax2.plot(Sub_df['Date'], Sub_df['WRF-S'], color='#FD4302', label='WRF-S')
    ax2.plot(Sub_df['Date'], Sub_df['WRF-H'], color='#0C87FF', label='WRF-H')
    # ax2.set_xlabel('Date')
    # ax2.set_ylabel('Value')
    ax2.legend(loc = 'upper right')
    add_TickGrid(ax2, IFminorOn=True)
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
    ax2.text(0.025, 0.95, f"({get_letter(YearIndex+1)})", fontsize=14, transform=ax2.transAxes, va='top')
    ax1.text(-0.8, 0.5, Year, fontsize=12, transform=ax2.transAxes, va='top', color='black')
    
plt.subplots_adjust(wspace=0.1,hspace=0.2)

metric_df = pd.DataFrame(results)
